In [2]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, Field
from typing import Dict, Any, List, Optional
import joblib
import numpy as np
import pandas as pd
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
import os
from datetime import datetime
import logging
from dotenv import load_dotenv

# Import your preprocessing functions
from preprocessing import SoilDataPreprocessor
from schema import WorkflowState

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
import pandas as pd

In [4]:
import os
import sys
from pathlib import Path

#sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), "..")))

# Get the project root directory
project_root = str(Path().absolute().parent)
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from models import ModelLoader
from agrovet import AgrovetLocator

loader = ModelLoader()
# Simple one-liner test
locator = AgrovetLocator.load_from_csv('/Users/oscar/Desktop/data-project/Fertiliser_Modelling/data/agrovet_data_cleaned.csv')

INFO:agrovet:Loaded agrovet data from /Users/oscar/Desktop/data-project/Fertiliser_Modelling/data/agrovet_data_cleaned.csv: 244 records
INFO:agrovet:Original columns: ['name                                ', ' lat         ', ' lon         ', ' products     ', ' prices']
INFO:agrovet:Normalized columns: ['name', 'lat', 'lon', 'products', 'prices']
INFO:agrovet:Mapped columns: {'name': 'name', 'lat': 'lat', 'lon': 'lon', 'products': 'products', 'prices': 'prices'}
INFO:agrovet:Final columns: ['name', 'lat', 'lon', 'products', 'prices']


In [ ]:
results = locator.find_nearest_agrovets(-1.2921, 36.8219, top_k=10, max_distance_km=100)
# show the results
print("Nearest Agrovets:")
df_results = pd.DataFrame(results)
df_results.head(10)

In [8]:
# Test with specific coordinates (example: Nairobi coordinates)
user_latitude = 0.4611
user_longitude = 34.1102

# user_latitude = 0.1580948
# user_longitude = 34.0982686

# Find nearest agrovets
results = locator.find_nearest_agrovets(
    user_lat=user_latitude,
    user_lon=user_longitude,
    top_k=5,  # Number of nearest agrovets to return
    max_distance_km=700 # Maximum search radius in km
)

# Display results
print(f"Found {len(results)} agrovets near coordinates ({user_latitude}, {user_longitude}):")
for i, agrovet in enumerate(results, 1):
    print(f"\n{i}. {agrovet['name']}")
    print(f"   Distance: {agrovet['distance_km']} km")
    print(f"   Location: ({agrovet['latitude']}, {agrovet['longitude']})")
    print(f"   Products: {', '.join(agrovet['products'])}")
    print(f"   Prices: {agrovet['prices']}")

print("\n" + "="*50)

Found 5 agrovets near coordinates (0.4611, 34.1102):

1. Hope Empire                         
   Distance: 0.12 km
   Location: (0.461285682, 34.11130629)
   Products: NPK, CAN, DAP
   Prices: [60.0, 55.0, 70.0]

2. Rajagrodealers                      
   Distance: 0.22 km
   Location: (0.462715923, 34.11138475)
   Products: NPK, CAN, DAP
   Prices: [60.0, 55.0, 70.0]

3. Multidose                           
   Distance: 0.41 km
   Location: (0.460094819, 34.11370419)
   Products: UREA, CAN
   Prices: [50.0, 55.0]

4. Multidose Agro Supplies Ltd         
   Distance: 0.45 km
   Location: (0.4601167, 34.1141333)
   Products: NPK, CAN, DAP
   Prices: [60.0, 55.0, 70.0]

5. Multidose Agro Supplies Ltd         
   Distance: 0.46 km
   Location: (0.4604002, 34.1142436)
   Products: NPK, CAN, DAP
   Prices: [60.0, 55.0, 70.0]



In [16]:
# Test with different coordinates (example: Mombasa coordinates)
mombasa_lat = -4.0435
mombasa_lon = 39.6682

results = locator.find_nearest_agrovets(
    user_lat=mombasa_lat,
    user_lon=mombasa_lon,
    top_k=5,
    max_distance_km=1000
)

print(f"Found {len(results)} agrovets near Mombasa:")
for agrovet in results:
    print(f"- {agrovet['name']}: {agrovet['distance_km']} km away")

print("\n" + "="*50)

Found 5 agrovets near Mombasa:
- Teso                                : 387.75 km away
- Nnnn                                : 387.77 km away
- Mary agrovet                        : 651.38 km away
- Aimad Enterprises                   : 763.23 km away
- Hybrid                              : 766.91 km away



In [18]:
# Method 3: Testing multiple locations
test_locations = [
    {"name": "Nairobi", "lat": -1.2921, "lon": 36.8219},
    {"name": "Kisumu", "lat": -0.1022, "lon": 34.7617},
    {"name": "Eldoret", "lat": 0.5143, "lon": 35.2698},
    {"name": "Nakuru", "lat": -0.3031, "lon": 36.0800}
]

for location in test_locations:
    print(f"\nTesting location: {location['name']}")
    results = locator.find_nearest_agrovets(
        user_lat=location['lat'],
        user_lon=location['lon'],
        top_k=3,
        max_distance_km=75
    )
    
    if results:
        nearest = results[0]
        print(f"Nearest agrovet: {nearest['name']} ({nearest['distance_km']} km)")
    else:
        print("No agrovets found in the area")

INFO:agrovet:No agrovets found within 75 km



Testing location: Nairobi
Nearest agrovet: Nnnn                                 (55.17 km)

Testing location: Kisumu
Nearest agrovet: Aimad Enterprises                    (65.36 km)

Testing location: Eldoret
Nearest agrovet: Mary agrovet                         (52.94 km)

Testing location: Nakuru
No agrovets found in the area
